In [1]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [2]:
gameid = 2021020814
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [3]:
# time.sleep(1200) #20 minutes
# time.sleep(2400) #40 minutes
# time.sleep(3600) #1 hour
# time.sleep(5400) #1.5 hours
# time.sleep(7200) #2 hours
# time.sleep(9000) #2.5 hours
# time.sleep(10800) #3 hours
# time.sleep(24000) #6 hours 40 minutes
# time.sleep(43200) #12 hours
time.sleep(111600)
# Use for delay start
pullurl = starturl + str(gameid) + endurl
pullurl
gameover = 'FALSE'
print(pullurl)

https://statsapi.web.nhl.com/api/v1/game/2021020814/feed/live.json


In [4]:
#Kings Ducks
while gameid <= 2021020917:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    if home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings':
        print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while gameover == "FALSE" and obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings'):
        while gameover == 'FALSE':
            #print(x)
            #x=x+1
            pullurl = starturl + str(gameid) + endurl
            try:
                with urllib.request.urlopen(pullurl) as url:
                    obj = json.loads(url.read().decode())
            except:
                print(f'The URL is broke, bruh. Try again.')
            allPlays = obj["liveData"]["plays"]['allPlays']
            hometeamscore = 0
            awayteamscore = 0
            for play in allPlays:
                period = play['about']['ordinalNum']
                event = play['result']['event']
                timeee = play['about']['periodTimeRemaining']
                description = play['result']['description']
                if event == 'Goal':
                    if str(home_team) == 'Los Angeles Kings':
                        if play['team']['name'] == 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                    if str(home_team) != 'Los Angeles Kings':
                        if play['team']['name'] != 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                    gamedetail.append(f'Period: {period}-{timeee}, {event}')
                    if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                        print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                        #Period Ready
                        #Giveaway
                        #Takeaway
                        #Hit
                    #elif event == 'Game Scheduled':
                        #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                        #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                        #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                        #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                        #time.sleep(8)
                    elif event == 'Period Start':
                        try:
                            api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                        #Period Start
                    elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                        try:
                            api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            if event == 'Game End':
                                gameover = 'TRUE'
                                gameused = gameid
                        except:
                            print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        time.sleep(8)
                        #Faceoff
                        #Blocked Shot
                        #Shot
                        #Penalty
                        #Missed Shot
                        #Stoppage
                        #Period Official
                        #Game End
                        #Game Official
                    elif event == 'Goal':
                        try:
                            api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                            print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        time.sleep(8)
                        #Goal
                    elif event == 'Period End' and (period == '1st' or period == '2nd'):
                        try:
                            api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(1080)
                        #Period End
                    elif event == 'Period End' and (period == '3rd'):
                        try:
                            api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    elif event == 'Period End' and (period == 'OT'):
                        try:
                            api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    else:
                        time.sleep(0)
            print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021020814-Final-2022-02-24-2022-02-25 -- Arizona Coyotes vs Los Angeles Kings
2021020828-Preview-2022-02-26-2022-02-25 -- Anaheim Ducks vs Los Angeles Kings
NOT TWEETED---Period: 1st-20:00, Game Scheduled-Game Scheduled
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
An

Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
A

Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
A

Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
A

Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
Anaheim Ducks-0, Los Angeles Kings-0
A

...

Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
A

Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
NOT TWEETED---Period: 3rd-04:52, Hit-Mikey Anderson hit Rickard Rakell
NOT 

Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
NOT TWEETED---Period: 3rd-03:56, Hit-Ryan Getzlaf hit Matt Roy
NOT TWEETED---Period: 3rd-02:59, Giveaway-Giveaway by Isac Lundestrom
NOT TWEETED---Period: 3rd-02:25, Hit-Trevor Zegras hi

Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Period: 3rd-02:16, Faceoff-Anze Kopitar faceoff won against Ryan Getzlaf #GoKingsGo
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles

Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
Anaheim Ducks-1, Los Angeles Kings-4
A

In [5]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [6]:
gameid = 2021020814
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [7]:
# time.sleep(1200) #20 minutes
# time.sleep(2400) #40 minutes
# time.sleep(3600) #1 hour
# time.sleep(5400) #1.5 hours
# time.sleep(7200) #2 hours
# time.sleep(9000) #2.5 hours
# time.sleep(10800) #3 hours
# time.sleep(24000) #6 hours 40 minutes
# time.sleep(43200) #12 hours
time.sleep(75600)
# Use for delay start
pullurl = starturl + str(gameid) + endurl
pullurl
gameover = 'FALSE'
print(pullurl)

https://statsapi.web.nhl.com/api/v1/game/2021020814/feed/live.json


In [8]:
#Kings Islanders
while gameid <= 2021020917:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    if home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings':
        print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while gameover == "FALSE" and obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings'):
        while gameover == 'FALSE':
            #print(x)
            #x=x+1
            pullurl = starturl + str(gameid) + endurl
            try:
                with urllib.request.urlopen(pullurl) as url:
                    obj = json.loads(url.read().decode())
            except:
                print(f'The URL is broke, bruh. Try again.')
            allPlays = obj["liveData"]["plays"]['allPlays']
            hometeamscore = 0
            awayteamscore = 0
            for play in allPlays:
                period = play['about']['ordinalNum']
                event = play['result']['event']
                timeee = play['about']['periodTimeRemaining']
                description = play['result']['description']
                if event == 'Goal':
                    if str(home_team) == 'Los Angeles Kings':
                        if play['team']['name'] == 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                    if str(home_team) != 'Los Angeles Kings':
                        if play['team']['name'] != 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                    gamedetail.append(f'Period: {period}-{timeee}, {event}')
                    if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                        print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                        #Period Ready
                        #Giveaway
                        #Takeaway
                        #Hit
                    #elif event == 'Game Scheduled':
                        #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                        #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                        #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                        #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                        #time.sleep(8)
                    elif event == 'Period Start':
                        try:
                            api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                        #Period Start
                    elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                        try:
                            api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            if event == 'Game End':
                                gameover = 'TRUE'
                                gameused = gameid
                        except:
                            print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        time.sleep(8)
                        #Faceoff
                        #Blocked Shot
                        #Shot
                        #Penalty
                        #Missed Shot
                        #Stoppage
                        #Period Official
                        #Game End
                        #Game Official
                    elif event == 'Goal':
                        try:
                            api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                            print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        time.sleep(8)
                        #Goal
                    elif event == 'Period End' and (period == '1st' or period == '2nd'):
                        try:
                            api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(1080)
                        #Period End
                    elif event == 'Period End' and (period == '3rd'):
                        try:
                            api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    elif event == 'Period End' and (period == 'OT'):
                        try:
                            api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    else:
                        time.sleep(0)
            print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021020814-Final-2022-02-24-2022-02-26 -- Arizona Coyotes vs Los Angeles Kings
2021020828-Final-2022-02-26-2022-02-26 -- Anaheim Ducks vs Los Angeles Kings
2021020837-Preview-2022-02-27-2022-02-26 -- Los Angeles Kings vs New York Islanders
NOT TWEETED---Period: 1st-20:00, Game Scheduled-Game Scheduled
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New 

Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Isla

Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Isla

Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Isla

Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Islanders-0
Los Angeles Kings-0, New York Isla

...

Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Period: 3rd-04:04, Faceoff-Anze Kopitar faceoff won against Brock Nelson #GoKingsGo
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Isla

Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Los Angeles Kings-4, New York Islanders-1
Period: 3rd-02:09, Blocked Shot-Brendan Lemieux shot blocked shot by Adam Pelech #GoKingsGo
NOT TWEETED---Period: 3rd-01:58, Giveaway-Giveaway by Matt Roy
NOT TWEETED---Period: 3rd-01:45, Hit-Blake Lizotte hit Brock Nelson
Los Angeles Kings-4, 

Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Islanders-2
Los Angeles Kings-4, New York Isla

Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Isla

Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Islanders-2
Los Angeles Kings-5, New York Isla

In [9]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [10]:
gameid = 2021020814
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'
auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [11]:
# time.sleep(1200) #20 minutes
# time.sleep(2400) #40 minutes
# time.sleep(3600) #1 hour
# time.sleep(5400) #1.5 hours
# time.sleep(7200) #2 hours
# time.sleep(9000) #2.5 hours
# time.sleep(10800) #3 hours
# time.sleep(24000) #6 hours 40 minutes
# time.sleep(43200) #12 hours
time.sleep(162000)
# Use for delay start
pullurl = starturl + str(gameid) + endurl
pullurl
gameover = 'FALSE'
print(pullurl)

https://statsapi.web.nhl.com/api/v1/game/2021020814/feed/live.json


In [12]:
#Kings Bruins
while gameid <= 2021020917:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    if home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings':
        print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while gameover == "FALSE" and obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings'):
        while gameover == 'FALSE':
            #print(x)
            #x=x+1
            pullurl = starturl + str(gameid) + endurl
            try:
                with urllib.request.urlopen(pullurl) as url:
                    obj = json.loads(url.read().decode())
            except:
                print(f'The URL is broke, bruh. Try again.')
            allPlays = obj["liveData"]["plays"]['allPlays']
            hometeamscore = 0
            awayteamscore = 0
            for play in allPlays:
                period = play['about']['ordinalNum']
                event = play['result']['event']
                timeee = play['about']['periodTimeRemaining']
                description = play['result']['description']
                if event == 'Goal':
                    if str(home_team) == 'Los Angeles Kings':
                        if play['team']['name'] == 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                    if str(home_team) != 'Los Angeles Kings':
                        if play['team']['name'] != 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                    gamedetail.append(f'Period: {period}-{timeee}, {event}')
                    if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                        print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                        #Period Ready
                        #Giveaway
                        #Takeaway
                        #Hit
                    #elif event == 'Game Scheduled':
                        #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                        #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                        #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                        #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                        #time.sleep(8)
                    elif event == 'Period Start':
                        try:
                            api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                        #Period Start
                    elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                        try:
                            api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            if event == 'Game End':
                                gameover = 'TRUE'
                                gameused = gameid
                        except:
                            print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        time.sleep(8)
                        #Faceoff
                        #Blocked Shot
                        #Shot
                        #Penalty
                        #Missed Shot
                        #Stoppage
                        #Period Official
                        #Game End
                        #Game Official
                    elif event == 'Goal':
                        try:
                            api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                            print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        time.sleep(8)
                        #Goal
                    elif event == 'Period End' and (period == '1st' or period == '2nd'):
                        try:
                            api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(1080)
                        #Period End
                    elif event == 'Period End' and (period == '3rd'):
                        try:
                            api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    elif event == 'Period End' and (period == 'OT'):
                        try:
                            api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    else:
                        time.sleep(0)
            print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021020814-Final-2022-02-24-2022-02-27 -- Arizona Coyotes vs Los Angeles Kings
2021020828-Final-2022-02-26-2022-02-27 -- Anaheim Ducks vs Los Angeles Kings
2021020837-Final-2022-02-27-2022-02-27 -- Los Angeles Kings vs New York Islanders
2021020850-Preview-2022-03-01-2022-02-27 -- Los Angeles Kings vs Boston Bruins
NOT TWEETED---Period: 1st-20:00, Game Scheduled-Game Scheduled
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston 

Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
L

Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
L

Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
L

Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
Los Angeles Kings-0, Boston Bruins-0
L

...

Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Period: 3rd-07:26, Missed Shot-Brad Marchand Missing key [ED_null] Cal Petersen #GoKingsGo
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, Boston Bruins-6
Los Angeles Kings-0, 

Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
L

Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
L

Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
L

Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
Los Angeles Kings-0, Boston Bruins-7
L